In [1]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
from sklearn.model_selection import train_test_split

In [2]:
file_path = './generated_dataset.csv'
data = pd.read_csv(file_path)
data.head()


,username,service_provider_username,service,price,no_of_pages,topic,date,deadline
0,user1,provider1,Website,1525,6,Topic1,2024-01-22,2024-02-02
1,user2,provider2,PPT,200,7,Topic2,2024-02-09,2024-02-14
2,user3,provider3,PPT,200,8,Topic3,2024-02-18,2024-03-02
3,user4,provider4,Website,1525,6,Topic4,2024-02-19,2024-03-01
4,user5,provider5,App,2075,6,Topic5,2024-02-12,2024-02-25


In [3]:
columns_to_keep = ['username', 'service_provider_username', 'service', 'price', 'no_of_pages', 'topic','date', 'deadline']
data = data[columns_to_keep].dropna()
data.head()



,username,service_provider_username,service,price,no_of_pages,topic,date,deadline
0,user1,provider1,Website,1525,6,Topic1,2024-01-22,2024-02-02
1,user2,provider2,PPT,200,7,Topic2,2024-02-09,2024-02-14
2,user3,provider3,PPT,200,8,Topic3,2024-02-18,2024-03-02
3,user4,provider4,Website,1525,6,Topic4,2024-02-19,2024-03-01
4,user5,provider5,App,2075,6,Topic5,2024-02-12,2024-02-25


In [4]:
y=data['price']
features=['service','no_of_pages','date','deadline']
X=data[features]
X.describe()

,no_of_pages
count,10000.00000
mean,7.78150
std,5.09619
min,1.00000
25%,4.00000
50%,6.00000
75%,10.25000
max,20.00000


In [5]:
data['date'] = pd.to_datetime(data['date'])
data['deadline'] = pd.to_datetime(data['deadline'])
data['days']=data['deadline']-data['date']



In [6]:
data.drop(['date','deadline'],axis=1,inplace=True)

In [7]:
data.drop(['username','service_provider_username'],axis=1,inplace=True)

In [8]:
data['days']=data['days'].dt.days
data.head()


,service,price,no_of_pages,topic,days
0,Website,1525,6,Topic1,11
1,PPT,200,7,Topic2,5
2,PPT,200,8,Topic3,13
3,Website,1525,6,Topic4,11
4,App,2075,6,Topic5,13


In [9]:
data.drop(['topic'],axis=1,inplace=True)

In [10]:
data

,service,price,no_of_pages,days
0,Website,1525,6,11
1,PPT,200,7,5
2,PPT,200,8,13
3,Website,1525,6,11
4,App,2075,6,13
...,...,...,...,...
9995,Website,1500,5,10
9996,Website,1500,3,15
9997,Website,1725,6,3
9998,Website,1500,5,13


In [11]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
y=data['price']
features=['service','no_of_pages','days']
X=data[features]
X.describe()
OrdinalEncoder=OrdinalEncoder()
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

lable_X_train=X_train.copy()
lable_X_test=X_test.copy()

lable_X_train[['service']]=OrdinalEncoder.fit_transform(X_train[['service']])
lable_X_test[['service']]=OrdinalEncoder.transform(X_test[['service']])

model=RandomForestRegressor(n_estimators=100,random_state=0)
print(lable_X_train)
print(lable_X_test)


      service  no_of_pages  days
2102      2.0           17     9
9448      2.0            2     5
462       3.0            5    10
4563      3.0            4     1
5130      1.0           20    10
...       ...          ...   ...
2773      1.0            5    11
1214      1.0            5     2
760       2.0            9     2
1346      3.0            5     8
9558      2.0           20    10

[8000 rows x 3 columns]
      service  no_of_pages  days
1428      2.0           14     4
5708      0.0            6     6
945       3.0            6    15
4050      2.0            9    10
4508      2.0            1    15
...       ...          ...   ...
6977      2.0            6    14
4833      2.0            1    14
3674      1.0            7     2
7450      0.0            3    10
2760      3.0            4     4

[2000 rows x 3 columns]


In [12]:
from sklearn.metrics import mean_absolute_error
def get_score(model,X_train,X_test,y_train,y_test):
    model.fit(X_train,y_train)
    preds=model.predict(X_test)
    return mean_absolute_error(y_test,preds)    

In [13]:
get_score(model,lable_X_train,lable_X_test,y_train,y_test)

0.013875

In [14]:
from sklearn.metrics import mean_squared_error
model.fit(lable_X_train,y_train)
preds=model.predict(lable_X_test)

accuracy=mean_squared_error(y_test,preds)
accuracy

0.01409375

In [15]:
from sklearn.metrics import r2_score
r2=r2_score(y_test,preds)
r2

0.9999999804941825

In [16]:
# 0->App
# 1->Docs
# 2->PPT
# 3->Website               
new_data = {
    "service":0,
    "no_of_pages":10,
    "days":4,
}

input_data_as_numpy_array = np.array([
    new_data["service"],new_data["no_of_pages"], new_data["days"]
]).reshape(1, -1)

prediction = model.predict(input_data_as_numpy_array)
print("Predicted Price:", prediction[0])


Predicted Price: 2400.0


d:\Python\Lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [17]:
# from sklearn.preprocessing import OrdinalEncoder
# from sklearn.ensemble import RandomForestRegressor
# y=data['price']
# features=['service','no_of_pages','days']
# X=data[features]
# custom_mapping={
#     'App':0,
#     'Docs':1,
#     'PPT':2,
#     'Website':3,
#     'Case Study':4,
#     'Figma':5,
#     'Frontend':6,
#     'Backend':7,
    
# }

In [18]:
# # New data for prediction
# new_data = {
#     "service_type": "PPT",  # Replace with the actual service type
#     "no_of_pages_screens": 6,
#     "days": 10,
# }

# # Perform ordinal encoding on the 'service_type' for new data
# encoded_service_type = ordinal_encoder.transform([[new_data["service_type"]]])[0][0]

# # Create input data as a DataFrame
# input_data = pd.DataFrame({
#     "service_type": [encoded_service_type],
#     "days": [new_data["days"]],
#     "no_of_pages_screens": [new_data["no_of_pages_screens"]],
# })

# # Predict using the best model obtained from GridSearchCV
# predicted_price = best_model.predict(input_data)

# # Decode the encoded service_type
# decoded_service_type = ordinal_encoder.inverse_transform([[encoded_service_type]])[0][0]

# print("Original service_type:", decoded_service_type)
# print("Predicted Price:", predicted_price[0])


In [19]:
# import pickle
# file_name="DynamicPrice.pkl"
# with open(file_name, 'wb') as file:
#     pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)
import joblib

joblib.dump(model, 'DynamicPrice.joblib')

['DynamicPrice.joblib']